In [ ]:

import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, RepeatedKFold, LeaveOneOut
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_predict
import numpy as np, seaborn as sns, matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import os
import sys

# Add project root to path (for Jupyter notebooks)
# Get the current directory and navigate to project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, project_root)

from app.config import Config
from app.services import EDA
from app.services import PresentValue

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
## FROM DATABASE
pv = PresentValue()
anual_increment = pv.fetch_incremento_from_database()

fase = "III"
preproccesing = EDA()
df_raw = preproccesing.assemble_projects_from_database(fase)
df_vp = preproccesing.create_dataset(pv.present_value_costs, fase=fase)

c:\Code\Road-Project-Prediction\src\eda.py:286: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  w = (df[cols] / totals).fillna(0)


In [44]:
def remove_outliers(df, target: str, method='ensemble', contamination=0.1, voting_threshold=0.5) -> pd.DataFrame:
    """
    Advanced outlier detection using multiple methods.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    target : str
        Target column name
    method : str
        'ensemble' (default): Combines multiple methods with voting
        'isolation_forest': Uses Isolation Forest only
        'lof': Uses Local Outlier Factor only
        'robust_statistical': Uses Modified Z-score with MAD
        'all_strict': All methods must agree (strictest)
    contamination : float
        Expected proportion of outliers (0.05-0.2 typical)
    voting_threshold : float
        For ensemble method, fraction of methods that must flag as outlier (0.5 = majority)
    
    Returns:
    --------
    pd.DataFrame
        Cleaned dataframe without outliers
    """
    from sklearn.ensemble import IsolationForest
    from sklearn.neighbors import LocalOutlierFactor
    from sklearn.preprocessing import StandardScaler
    
    # Remove zero values first (domain-specific)
    df_nonzero = df[df[target] != 0].copy()
    
    if len(df_nonzero) < 10:
        print(f"Warning: Only {len(df_nonzero)} samples. Skipping outlier detection.")
        return df_nonzero
    
    # Prepare features: numerical columns + target
    numerical_cols = df_nonzero.select_dtypes(include=[np.number]).columns.tolist()
    
    # Scale features for better outlier detection
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_nonzero[numerical_cols])
    
    # Initialize outlier flags
    outlier_flags = {}
    
    # Method 1: Isolation Forest (excellent for high-dimensional data)
    if method in ['ensemble', 'isolation_forest', 'all_strict']:
        iso_forest = IsolationForest(
            contamination=contamination,
            random_state=42,
            n_estimators=200,
            max_samples='auto',
            bootstrap=True
        )
        iso_predictions = iso_forest.fit_predict(X_scaled)
        outlier_flags['isolation_forest'] = (iso_predictions == -1)
        
    # Method 2: Local Outlier Factor (density-based, good for local anomalies)
    if method in ['ensemble', 'lof', 'all_strict']:
        n_neighbors = min(20, len(df_nonzero) - 1)
        lof = LocalOutlierFactor(
            n_neighbors=n_neighbors,
            contamination=contamination
        )
        lof_predictions = lof.fit_predict(X_scaled)
        outlier_flags['lof'] = (lof_predictions == -1)
    
    # Method 3: Modified Z-score with MAD (robust to outliers themselves)
    if method in ['ensemble', 'robust_statistical', 'all_strict']:
        target_values = df_nonzero[target].values
        median = np.median(target_values)
        mad = np.median(np.abs(target_values - median))
        
        # Modified Z-score (more robust than standard Z-score)
        if mad != 0:
            modified_z_scores = 0.6745 * (target_values - median) / mad
            # Threshold of 3.5 is standard for modified Z-score
            outlier_flags['robust_statistical'] = np.abs(modified_z_scores) > 3.5
        else:
            outlier_flags['robust_statistical'] = np.zeros(len(df_nonzero), dtype=bool)
    
    # Method 4: Multivariate Z-score on target (additional check)
    if method in ['ensemble', 'all_strict']:
        target_scaled = scaler.fit_transform(df_nonzero[[target]])
        outlier_flags['z_score'] = np.abs(target_scaled.flatten()) > 3
    
    # Combine methods based on selected strategy
    if method == 'ensemble':
        # Voting: flag as outlier if voting_threshold fraction of methods agree
        outlier_matrix = np.column_stack(list(outlier_flags.values()))
        votes = outlier_matrix.sum(axis=1)
        is_outlier = votes >= (len(outlier_flags) * voting_threshold)
        
    elif method == 'all_strict':
        # All methods must agree (most conservative)
        outlier_matrix = np.column_stack(list(outlier_flags.values()))
        is_outlier = outlier_matrix.all(axis=1)
        
    elif method in ['isolation_forest', 'lof', 'robust_statistical']:
        # Single method
        is_outlier = outlier_flags[method]
    
    else:
        raise ValueError(f"Unknown method: {method}")
    
    # Filter out outliers
    df_clean = df_nonzero[~is_outlier].copy()
    
    # Report
    n_outliers = is_outlier.sum()
    pct_removed = (n_outliers / len(df_nonzero)) * 100
    print(f"  → Removed {n_outliers}/{len(df_nonzero)} outliers ({pct_removed:.1f}%) using {method}")
    
    if method == 'ensemble' and len(outlier_flags) > 1:
        for flag_name, flags in outlier_flags.items():
            print(f"     • {flag_name}: {flags.sum()} outliers")
    
    return df_clean

In [45]:
def rmsle_scorer(y_true, y_pred):
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true))**2))

def train_model(df_clean, predictor_name, hue_name, target_name):
    X = df_clean[[predictor_name, hue_name]].copy()
    X[predictor_name + ' LOG'] = np.log1p(X[predictor_name])
    y = df_clean[target_name].astype(float)

    pre = ColumnTransformer([
        ('num', StandardScaler(), [predictor_name, predictor_name + ' LOG']),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), [hue_name])
    ])

    svr = SVR(kernel='rbf')
    pipe = Pipeline([('pre', pre), ('svr', svr)])
    model = TransformedTargetRegressor(regressor=pipe, func=np.log1p, inverse_func=np.expm1)

    param_grid = {
        'regressor__svr__C': [5, 10, 80, 200, 1000],
        'regressor__svr__epsilon': [0.01],
        'regressor__svr__gamma': ['scale', 'auto', 0.01, 0.1, 1.0],
    }

    cv = RepeatedKFold(n_splits=min(5, len(y)//2), n_repeats=min(5, len(y)//2), random_state=42) if len(y) >= 10 else LeaveOneOut()
    gs = GridSearchCV(model, param_grid, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, refit=True)
    gs.fit(X, y)


    cv_simple = RepeatedKFold(n_splits=min(5, len(y)//2), n_repeats=1, random_state=42) if len(y) >= 10 else LeaveOneOut()
    y_oof = cross_val_predict(gs.best_estimator_, X, y, cv=cv_simple, n_jobs=-1)
    
    r2_oof = r2_score(y, y_oof)
    mae_oof = mean_absolute_error(y, y_oof)
    rmse_oof = np.sqrt(mean_squared_error(y, y_oof))
    rmsle_oof = rmsle_scorer(y, y_oof)
    mape_oof = np.mean(np.abs((y - y_oof) / y.replace(0, np.nan))) * 100

    print('Best params:', gs.best_params_)
    print({'R2': r2_oof, 'MAE': mae_oof, 'RMSE': rmse_oof, 'RMSLE': rmsle_oof, 'MAPE%': float(mape_oof)})
    
    y_predicted = y_oof
   
    return X, y, y_predicted, gs.best_estimator_
    

In [54]:
def analysis_plots(y, y_predicted, df_item_cleaned, predictor_name, target_name, hue_name):
    """
    Creates beautiful executive Plotly visualizations for model analysis
    
    Parameters:
    - y: Actual values
    - y_predicted: Predicted values
    - df_item_cleaned: Cleaned dataframe with all project data
    - predictor_name: Name of the predictor column (e.g., 'LONGITUD KM')
    - target_name: Name of the target column (e.g., '5 - TALUDES')
    - hue_name: Name of the hue column (e.g., 'ALCANCE')
    """
    
    # Get df_raw to access project codes and names
    global df_raw
    
    # Create hover data with project information
    hover_data = []
    for idx in df_item_cleaned.index:
        hover_text = ""
        # Try to get project code and name from df_raw
        if 'df_raw' in globals() and idx in df_raw.index:
            if 'CÓDIGO DEL PROYECTO' in df_raw.columns:
                hover_text += f"<b>Código:</b> {df_raw.loc[idx, 'CÓDIGO DEL PROYECTO']}<br>"
            if 'NOMBRE DEL PROYECTO' in df_raw.columns:
                hover_text += f"<b>Nombre:</b> {df_raw.loc[idx, 'NOMBRE DEL PROYECTO']}<br>"
        
        if predictor_name in df_item_cleaned.columns:
            hover_text += f"<b>{predictor_name}:</b> {df_item_cleaned.loc[idx, predictor_name]:.2f}<br>"
        if hue_name in df_item_cleaned.columns:
            hover_text += f"<b>{hue_name}:</b> {df_item_cleaned.loc[idx, hue_name]}<br>"
        hover_data.append(hover_text)
    
    # Plot 1: Actual vs Predicted (Plotly)
    fig1 = go.Figure()
    
    # Add scatter points colored by Alcance
    colors_map = {
        'Segunda calzada': '#1f77b4',
        'operacion y mantenimiento': '#ff7f0e', 
        'Mejoramiento': '#2ca02c',
        'Rehabilitación': '#d62728',
        'Nuevo': '#9467bd',
        'Construcción': '#8c564b',
        'Puesta a punto': '#e377c2'
    }
    
    # Convert y and y_predicted to pandas Series if they aren't already
    if not isinstance(y, pd.Series):
        y = pd.Series(y, index=df_item_cleaned.index)
    if not isinstance(y_predicted, pd.Series):
        y_predicted = pd.Series(y_predicted, index=df_item_cleaned.index)
    
    for alcance_type in sorted(df_item_cleaned[hue_name].unique()):
        mask = df_item_cleaned[hue_name] == alcance_type
        indices = df_item_cleaned[mask].index
        y_actual_subset = y[mask]
        y_pred_subset = y_predicted[mask]
        
        hover_subset = [hover_data[list(df_item_cleaned.index).index(idx)] + 
                       f"<b>Valor Real:</b> ${y_actual_subset.loc[idx]:,.0f}<br><b>Predicción:</b> ${y_pred_subset.loc[idx]:,.0f}"
                       for idx in indices]
        
        fig1.add_trace(go.Scatter(
            x=y_actual_subset,
            y=y_pred_subset,
            mode='markers',
            name=alcance_type,
            marker=dict(
                size=12,
                color=colors_map.get(alcance_type, '#7f7f7f'),
                opacity=0.8,
                line=dict(width=1, color='white')
            ),
            hovertemplate='%{customdata}<extra></extra>',
            customdata=hover_subset
        ))
    
    # Add perfect prediction line
    min_val = min(y.min(), y_predicted.min())
    max_val = max(y.max(), y_predicted.max())
    fig1.add_trace(go.Scatter(
        x=[min_val, max_val],
        y=[min_val, max_val],
        mode='lines',
        name='Predicción Perfecta',
        line=dict(color='red', width=2, dash='dash'),
        showlegend=True
    ))
    
    # Update layout for executive look
    fig1.update_layout(
        title=dict(
            text=f'<b>Predicción vs Realidad - {target_name}</b>',
            x=0.5,
            xanchor='center',
            font=dict(size=20, family='Arial Black')
        ),
        xaxis=dict(
            title=dict(
                text='<b>Valor Real ($)</b>',
                font=dict(size=14)
            ),
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=0.5
        ),
        yaxis=dict(
            title=dict(
                text='<b>Valor Predicho ($)</b>',
                font=dict(size=14)
            ),
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=0.5
        ),
        plot_bgcolor='white',
        paper_bgcolor='white',
        hovermode='closest',
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="lightgray",
            borderwidth=1
        ),
        width=900,
        height=600
    )
    
    fig1.show()

In [ ]:
def train_and_calculate_metrics(df, target_columns, predictor_name, hue_name):
    results = {}
    
    for target_name in target_columns:
        df_item = df.loc[:, [predictor_name, hue_name, target_name]]
        df_item_cleaned = remove_outliers(df_item, target_name) 
        # df_item_cleaned = remove_outliers(df_item, target_name, method='all_strict') 
        # preproccesing.show_plots_eda(predictor_name, target_name, hue_name, df_item_cleaned)
        print(target_name)
        X, y, y_predicted, trained_model = train_model(df_item_cleaned, predictor_name, hue_name, target_name)
        analysis_plots(y, y_predicted, df_item_cleaned, predictor_name, target_name, hue_name)
        
        results[target_name] = {
            'X': X,
            'y': y,
            'y_predicted': y_predicted,
            'trained_model': trained_model
        }
        
        # if target_name == '16 - DIRECCIÓN Y COORDINACIÓN':
        #     break
    return results    

predictor_name = 'LONGITUD KM'
hue_name = 'ALCANCE'

# ['LONGITUD KM', 'ALCANCE', '1 - TRANSPORTE', '2.1 - INFORMACIÓN GEOGRÁFICA', '2.2 - TRAZADO Y DISEÑO GEOMÉTRICO',
#        '2.3 - SEGURIDAD VIAL', '2.4 - SISTEMAS INTELIGENTES', '3.1 - GEOLOGÍA','3.2 - HIDROGEOLOGÍA', '4 - SUELOS', '5 - TALUDES', '6 - PAVIMENTO',
#        '7 - SOCAVACIÓN', '8 - ESTRUCTURAS', '9 - TÚNELES', '10 - URBANISMO Y PAISAJISMO', '11 - PREDIAL', '12 - IMPACTO AMBIENTAL',
#        '13 - CANTIDADES', '14 - EVALUACIÓN SOCIOECONÓMICA', '15 - OTROS - MANEJO DE REDES', '16 - DIRECCIÓN Y COORDINACIÓN']

target_columns = ['1 - TRANSPORTE', '2.1 - INFORMACIÓN GEOGRÁFICA','2.2 - TRAZADO Y DISEÑO GEOMÉTRICO', '2.3 - SEGURIDAD VIAL', '2.4 - SISTEMAS INTELIGENTES', 
                  '5 - TALUDES', '6 - PAVIMENTO', '7 - SOCAVACIÓN', '11 - PREDIAL', 
                  '12 - IMPACTO AMBIENTAL', '15 - OTROS - MANEJO DE REDES', '16 - DIRECCIÓN Y COORDINACIÓN']

df = df_vp[['LONGITUD KM', 'ALCANCE', 'TIPO TERRENO']].join(df_vp.loc[:, '1 - TRANSPORTE':'16 - DIRECCIÓN Y COORDINACIÓN'])
results = train_and_calculate_metrics(df, target_columns, predictor_name, hue_name)    

1 - TRANSPORTE
Best params: {'regressor__svr__C': 5, 'regressor__svr__epsilon': 0.01, 'regressor__svr__gamma': 'scale'}
{'R2': -2.999999999999962, 'MAE': 4643975.471230011, 'RMSE': np.float64(4643975.471230011), 'RMSLE': np.float64(0.23571981522235674), 'MAPE%': 23.790881617359197}


  → Removed 1/10 outliers (10.0%) using ensemble
     • isolation_forest: 1 outliers
     • lof: 1 outliers
     • robust_statistical: 4 outliers
     • z_score: 0 outliers
2.1 - INFORMACIÓN GEOGRÁFICA
Best params: {'regressor__svr__C': 1000, 'regressor__svr__epsilon': 0.01, 'regressor__svr__gamma': 0.01}
{'R2': 0.9997796987954439, 'MAE': 199650.3645912337, 'RMSE': np.float64(262145.3416134265), 'RMSLE': np.float64(0.09552108481900061), 'MAPE%': 5.977188326771584}


  → Removed 5/50 outliers (10.0%) using ensemble
     • isolation_forest: 5 outliers
     • lof: 5 outliers
     • robust_statistical: 3 outliers
     • z_score: 2 outliers
2.2 - TRAZADO Y DISEÑO GEOMÉTRICO
Best params: {'regressor__svr__C': 10, 'regressor__svr__epsilon': 0.01, 'regressor__svr__gamma': 'auto'}
{'R2': 0.8262669836001061, 'MAE': 2270169.4857858047, 'RMSE': np.float64(5467999.962276422), 'RMSLE': np.float64(0.38350201315039295), 'MAPE%': 22.301882321811455}


  → Removed 4/49 outliers (8.2%) using ensemble
     • isolation_forest: 5 outliers
     • lof: 5 outliers
     • robust_statistical: 2 outliers
     • z_score: 1 outliers
2.3 - SEGURIDAD VIAL
Best params: {'regressor__svr__C': 5, 'regressor__svr__epsilon': 0.01, 'regressor__svr__gamma': 'scale'}
{'R2': 0.7918920978029302, 'MAE': 2511792.6589240604, 'RMSE': np.float64(4750779.3538916), 'RMSLE': np.float64(0.49436419068004045), 'MAPE%': 35.99627260044706}


  → Removed 5/42 outliers (11.9%) using ensemble
     • isolation_forest: 5 outliers
     • lof: 5 outliers
     • robust_statistical: 4 outliers
     • z_score: 1 outliers
2.4 - SISTEMAS INTELIGENTES
Best params: {'regressor__svr__C': 200, 'regressor__svr__epsilon': 0.01, 'regressor__svr__gamma': 0.01}
{'R2': 0.8512799272969779, 'MAE': 409025.9121887115, 'RMSE': np.float64(1723149.2436338912), 'RMSLE': np.float64(0.12688379723284127), 'MAPE%': 4.590438781928622}


...

  → Removed 5/50 outliers (10.0%) using ensemble
     • isolation_forest: 5 outliers
     • lof: 5 outliers
     • robust_statistical: 1 outliers
     • z_score: 1 outliers
7 - SOCAVACIÓN
Best params: {'regressor__svr__C': 5, 'regressor__svr__epsilon': 0.01, 'regressor__svr__gamma': 'scale'}
{'R2': 0.9191092259281675, 'MAE': 3759520.04480352, 'RMSE': np.float64(6367428.417797689), 'RMSLE': np.float64(0.28881793052083904), 'MAPE%': 19.058232143082638}


11 - PREDIAL
Best params: {'regressor__svr__C': 5, 'regressor__svr__epsilon': 0.01, 'regressor__svr__gamma': 1.0}
{'R2': -0.4044402705078842, 'MAE': 710990.1232883679, 'RMSE': np.float64(1052513.0803657123), 'RMSLE': np.float64(0.059109258098428374), 'MAPE%': 3.8924951032041744}


12 - IMPACTO AMBIENTAL
Best params: {'regressor__svr__C': 5, 'regressor__svr__epsilon': 0.01, 'regressor__svr__gamma': 'scale'}
{'R2': -3.000000000000001, 'MAE': 117199626.3808198, 'RMSE': np.float64(117199626.38081981), 'RMSLE': np.float64(4.2750618423999285), 'MAPE%': 3593.535818986291}


  → Removed 4/34 outliers (11.8%) using ensemble
     • isolation_forest: 4 outliers
     • lof: 4 outliers
     • robust_statistical: 7 outliers
     • z_score: 1 outliers
15 - OTROS - MANEJO DE REDES
Best params: {'regressor__svr__C': 200, 'regressor__svr__epsilon': 0.01, 'regressor__svr__gamma': 0.1}
{'R2': 0.7624701568730734, 'MAE': 3693765.6476776865, 'RMSE': np.float64(8049804.631489442), 'RMSLE': np.float64(0.3844994149087249), 'MAPE%': 20.865049176069167}


  → Removed 6/51 outliers (11.8%) using ensemble
     • isolation_forest: 5 outliers
     • lof: 5 outliers
     • robust_statistical: 9 outliers
     • z_score: 3 outliers
16 - DIRECCIÓN Y COORDINACIÓN
Best params: {'regressor__svr__C': 10, 'regressor__svr__epsilon': 0.01, 'regressor__svr__gamma': 'scale'}
{'R2': 0.21370390700900155, 'MAE': 8699400.727068625, 'RMSE': np.float64(26692933.321417972), 'RMSLE': np.float64(0.6642193520187814), 'MAPE%': 27.372159165142275}


In [19]:
# ['LONGITUD KM', 'ALCANCE', '1 - TRANSPORTE', '2.1 - INFORMACIÓN GEOGRÁFICA', '2.2 - TRAZADO Y DISEÑO GEOMÉTRICO',
#        '2.3 - SEGURIDAD VIAL', '2.4 - SISTEMAS INTELIGENTES', '3.1 - GEOLOGÍA','3.2 - HIDROGEOLOGÍA', '4 - SUELOS', '5 - TALUDES', '6 - PAVIMENTO',
#        '7 - SOCAVACIÓN', '8 - ESTRUCTURAS', '9 - TÚNELES', '10 - URBANISMO Y PAISAJISMO', '11 - PREDIAL', '12 - IMPACTO AMBIENTAL',
#        '13 - CANTIDADES', '14 - EVALUACIÓN SOCIOECONÓMICA', '15 - OTROS - MANEJO DE REDES', '16 - DIRECCIÓN Y COORDINACIÓN']

df_items = df_vp[['LONGITUD KM', 'ALCANCE']].join(df_vp.loc[:, '1 - TRANSPORTE':])
df_items
# remove_outliers(df_items, '2.1 - INFORMACIÓN GEOGRÁFICA')

,LONGITUD KM,ALCANCE,1 - TRANSPORTE,2.1 - INFORMACIÓN GEOGRÁFICA,2.2 - TRAZADO Y DISEÑO GEOMÉTRICO,2.3 - SEGURIDAD VIAL,2.4 - SISTEMAS INTELIGENTES,3.1 - GEOLOGÍA,3.2 - HIDROGEOLOGÍA,4 - SUELOS,...,7 - SOCAVACIÓN,8 - ESTRUCTURAS,9 - TÚNELES,10 - URBANISMO Y PAISAJISMO,11 - PREDIAL,12 - IMPACTO AMBIENTAL,13 - CANTIDADES,14 - EVALUACIÓN SOCIOECONÓMICA,15 - OTROS - MANEJO DE REDES,16 - DIRECCIÓN Y COORDINACIÓN
0,17.400,Segunda calzada,0.000000e+00,0.000000e+00,1.353405e+08,7.287564e+07,4.013575e+07,1.239362e+08,0.000000e+00,1.878437e+08,...,5.275975e+07,6.838215e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.377238e+07,0.000000e+00,5.027965e+07,1.714043e+08
1,13.690,Nuevo,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.443049e+07,6.507553e+07,0.000000e+00,1.937793e+08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.634289e+08
2,10.000,Segunda calzada,0.000000e+00,1.332293e+06,6.119988e+06,1.619942e+06,6.750908e+06,0.000000e+00,0.000000e+00,0.000000e+00,...,1.240165e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.825669e+06,3.505045e+06
3,14.300,Segunda calzada,0.000000e+00,1.905179e+06,8.751583e+06,2.316517e+06,9.653799e+06,0.000000e+00,0.000000e+00,0.000000e+00,...,1.773436e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.760706e+06,5.012214e+06
4,10.200,operacion y mantenimiento,0.000000e+00,1.358939e+06,6.242388e+06,1.652341e+06,6.885926e+06,0.000000e+00,0.000000e+00,0.000000e+00,...,1.264968e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.962182e+06,3.575146e+06
5,8.500,operacion y mantenimiento,0.000000e+00,1.132449e+06,5.201990e+06,1.376951e+06,5.738272e+06,0.000000e+00,0.000000e+00,0.000000e+00,...,1.054140e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.801818e+06,2.979288e+06
6,13.500,Segunda calzada,0.000000e+00,1.798596e+06,8.261984e+06,2.186922e+06,9.113726e+06,0.000000e+00,0.000000e+00,0.000000e+00,...,1.674223e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.214653e+06,4.731811e+06
7,47.200,operacion y mantenimiento,0.000000e+00,6.288425e+06,2.888634e+07,7.646127e+06,3.186429e+07,0.000000e+00,0.000000e+00,0.000000e+00,...,5.853579e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.221716e+07,1.654381e+07
8,7.600,Segunda calzada,0.000000e+00,1.012543e+06,4.651191e+06,1.231156e+06,5.130690e+06,0.000000e+00,0.000000e+00,0.000000e+00,...,9.425254e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.187508e+06,2.663834e+06
9,5.000,Mejoramiento,0.000000e+00,0.000000e+00,6.311029e+06,2.105373e+06,0.000000e+00,1.385817e+07,0.000000e+00,1.326106e+07,...,6.289853e+06,6.421774e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.901125e+07,6.672842e+06


In [ ]:
# Extract the last target's results (16 - DIRECCIÓN Y COORDINACIÓN)
last_target = '16 - DIRECCIÓN Y COORDINACIÓN'
X = results[last_target]['X']
y = results[last_target]['y']
y_predicted = results[last_target]['y_predicted']

X['ACTUAL'] = y
X['PREDICTED'] = y_predicted
output = X.drop(columns=['LONGITUD KM LOG'])

# Calculate various accuracy metrics for each prediction
output['APE (%)'] = (abs(X['ACTUAL'] - X['PREDICTED']) / X['ACTUAL'].replace(0, np.nan)) * 100  # Absolute Percentage Error
output['ACCURACY (%)'] = 100 - output['APE (%)']  # Accuracy as percentage

# Add quality indicators
output['WITHIN_20%'] = output['APE (%)'] <= 20  # Flag for acceptable predictions

# Display summary statistics
print("PREDICTION ACCURACY SUMMARY")
print(f"Mean Absolute Percentage Error (MAPE): {output['APE (%)'].mean():.2f}%")
print(f"Median Absolute Percentage Error: {output['APE (%)'].median():.2f}%")
print(f"Mean Accuracy: {output['ACCURACY (%)'].mean():.2f}%")
print(f"Predictions within ±20%: {output['WITHIN_20%'].sum()} / {len(output)} ({output['WITHIN_20%'].sum()/len(output)*100:.1f}%)")

# Sort by absolute percentage error (worst predictions first) and display
output.sort_values(by='APE (%)', ascending=False)

PREDICTION ACCURACY SUMMARY
Mean Absolute Percentage Error (MAPE): 21.41%
Median Absolute Percentage Error: 4.19%
Mean Accuracy: 78.59%
Predictions within ±20%: 25 / 37 (67.6%)


,LONGITUD KM,ALCANCE,ACTUAL,PREDICTED,APE (%),ACCURACY (%),WITHIN_20%
19,9.86,Mejoramiento,4.078240e+06,8.088686e+06,98.337683,1.662317,False
9,5.00,Mejoramiento,2.573660e+07,2.072171e+06,91.948543,8.051457,False
24,10.06,Mejoramiento,4.427665e+07,8.204297e+06,81.470375,18.529625,False
23,5.00,Mejoramiento,6.931841e+06,2.072171e+06,70.106484,29.893516,False
20,8.35,Rehabilitación,7.111403e+07,2.171703e+07,69.461685,30.538315,False
18,14.52,Mejoramiento,6.005683e+06,9.764899e+06,62.594305,37.405695,False
16,6.56,Mejoramiento,2.713312e+06,4.390557e+06,61.815448,38.184552,False
29,0.80,Construcción,2.032312e+06,9.285025e+05,54.313000,45.687000,False
32,19.31,Puesta a punto,4.905494e+07,3.358585e+07,31.534214,68.465786,False
49,15.76,Segunda calzada,5.242439e+07,3.733342e+07,28.786173,71.213827,False
